In [1]:
import os, glob, base64
import datetime
import pandas as pd
import concurrent.futures
import google.generativeai as genai
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from dotenv import find_dotenv, load_dotenv
import json
import yfinance as yf

c:\Users\Max\Desktop\projects\quanticon\pyquant\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
today_str = datetime.datetime.today().strftime('%Y-%m-%d')

# Configuration
SHEET_ID = "15IfaN1fei9P6BXt0Nj7Rdj7SedDoN_Puzgyb6gUboVQ"
SHEET_NAME = "Sheet1"
DEFAULT_MODEL = "gemini-2.5-flash-preview-05-20"

# Load environment variables from .env file
dotenv_path = find_dotenv()
load_dotenv(dotenv_path)

# Check if GEMINI_API_KEY is loaded
if not os.getenv("GEMINI_API_KEY"):
    raise ValueError("GEMINI_API_KEY not found in .env file")

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [14]:
def get_trade_ideas():
    today_minus_day = datetime.datetime.today() - datetime.timedelta(1)
    today_minus_day_str = today_minus_day.strftime('%Y-%m-%d')
    print(today_minus_day)

    f_name1 = f'./outputs/{today_str}/backtest_ideas.csv'
    f_name2 = f'./outputs/{today_minus_day_str}/backtest_ideas.csv'
    if os.path.isfile(f_name1): 
        ideas = pd.read_csv(f_name1)
        # return ideas
    elif os.path.isfile(f_name2):
        ideas = pd.read_csv(f_name2)
        # return ideas
    else:
        try:
            scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
            print("setting creds...")
            creds = ServiceAccountCredentials.from_json_keyfile_name(
                os.getenv("GOOGLE_SHEET_API_KEY"), scope
            )
            print('authorizing...')
            client = gspread.authorize(creds)
            print("opening spreadsheet...")
            # sheet = client.open_by_key(SHEET_ID).worksheet(SHEET_NAME)
            spreadsheet = client.open_by_key(SHEET_ID)
            print("opening sheet...")

            sheet = spreadsheet.worksheet(SHEET_NAME)
            sheet_values = sheet.get_all_values()
            ideas_raw = pd.DataFrame(sheet_values[1:], columns=sheet_values[0])
            ideas_raw['date'] = pd.to_datetime(ideas_raw['date'])
            ideas = ideas_raw.loc[ideas_raw['date']<today_minus_day]
            ideas = ideas.loc[(ideas['date of outcome'].isna())|(ideas['date of outcome']=='')]
            # print(ideas)
            ideas.to_csv(f'./outputs/{today_str}/backtest_ideas.csv', index=False)
        except Exception as e:
            print(f'Error getting sheet: {e}')
            ideas = pd.DataFrame()
    
    return ideas

def extract_last_60_days(ticker):
    ticker_data = yf.download(ticker, period='1y', interval='1d', group_by='ticker')
    df = ticker_data.tail(60)
    return df

def csv_to_base64(df:pd.DataFrame):
    csv_string = df.to_string()
    return csv_string

def call_idea_parse_agent(idea):
    # print('configuring gemini...')
    # api_key = os.getenv("GEMINI_API_KEY")
    # genai.configure(api_key=api_key)
    # print('gemini configured. init gemini...')
    llm = ChatGoogleGenerativeAI(model=DEFAULT_MODEL, temperature=0, api_key=GEMINI_API_KEY)

    system_message = '''You are a helpful text parsing assistant. Your outputs are always in json, and you never provide any extra commentary beyond '''
    system_message += '''what the user requests. It's critical that your ouput is always json, as it's meant to be consumed later by other APIs.'''
    user_message = f"""{idea}\n\nAnalyze this trade idea and parse the ticker (ticker only, no special characters allowed), the entry, """
    user_message += """the stop loss, and the target. Your output should simply be a json object like so: \n\n"""
    user_message += """{{"ticker":<parsed ticker>, "entry":<parsed entry>, "stop loss":<parsed stop loss>, "target":<parsed target>}} \n\n"""
    user_message += """There should be absolutely no other commentary, only the requested information."""
    # print('starting messages')
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_message),
        ("user", user_message)
    ])

    # print('created messages')
    chain = prompt | llm
    response = chain.invoke({"idea": idea})
    # print('invoked chain')
    return response.content

def call_analysis_agent(idea, date):

    idea_obj_raw = call_idea_parse_agent(idea)
    idea_obj = idea_obj_raw.replace('```json','').replace('```','')
    json_idea = json.loads(idea_obj)

    price_history = extract_last_60_days(json_idea['ticker'])
    prices = price_history[json_idea['ticker']].reset_index()
    recent_prices = prices.loc[prices['Date']>date]
    recent_prices['entry'] = json_idea['entry']
    recent_prices['sl'] = json_idea['stop loss']
    recent_prices['tp'] = json_idea['target']
    recent_prices['direction'] = 'bull' if json_idea['target'] > json_idea['entry'] else 'bear'
    
    # entries
    recent_prices['trade_entered'] = 0
    ef1 = recent_prices['direction'] == 'bull'
    ef2 = recent_prices['High'] >= recent_prices['entry']
    ef3 = recent_prices['direction'] == 'bear'
    ef4 = recent_prices['Low'] <= recent_prices['entry']
    recent_prices['trade_entered'] = recent_prices['trade_entered'].mask(((ef1)&(ef2))|((ef3)&(ef4)),1)
    
    # SLs
    recent_prices['sl_hit'] = 0
    sf1 = recent_prices['direction'] == 'bull'
    sf2 = recent_prices['Low'] <= recent_prices['sl']
    sf3 = recent_prices['direction'] == 'bear'
    sf4 = recent_prices['High'] >= recent_prices['sl']
    recent_prices['sl_hit'] = recent_prices['sl_hit'].mask(((sf1)&(sf2))|((sf3)&(sf4)),1)

    # TPs
    recent_prices['tp_hit'] = 0
    tf1 = recent_prices['direction'] == 'bull'
    tf2 = recent_prices['High'] >= recent_prices['tp']
    tf3 = recent_prices['direction'] == 'bear'
    tf4 = recent_prices['Low'] <= recent_prices['tp']
    recent_prices['tp_hit'] = recent_prices['tp_hit'].mask(((tf1)&(tf2))|((tf3)&(tf4)),1)

    analysis_gb = recent_prices.groupby(['trade_entered', 'sl_hit', 'tp_hit']).agg({'Date':min})

    price_history_str = csv_to_base64(analysis_gb)

    llm = ChatGoogleGenerativeAI(model=DEFAULT_MODEL, temperature=0, api_key=GEMINI_API_KEY)

    system_message = '''You are a helpful stock backtest analysis assistant. Your outputs are always in json, and you never provide any extra commentary beyond '''
    system_message += '''what the user requests. It's critical that your ouput is always json, as it's meant to be consumed later by other APIs.'''
    user_message = f"""<idea>{json_idea}</idea><price summary>{price_history_str}</price summary>\n\n"""
    user_message = f"""Analyze this trade idea, along with the pricing summary, and tell me if 1 of the following occurred: - TP was hit\n- SL """
    user_message += """was hit\n- the trade is still going\n- never entered trade\n\nAlso, please tell me the date that this occurred on."""
    user_message += """There should be absolutely no other commentary, only the requested information."""
    user_message += """Your output should be in the following json format: \n\n"""
    user_message += """{{"outcome":<simple response>, "date of outcome": <YYYY-MM-DD for date outcome occurred>}} """
    # print('starting messages')
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_message),
        ("user", user_message)
    ])

    # print('created messages')
    chain = prompt | llm
    response = chain.invoke({"price_history_str": price_history_str, "json_idea":json.dumps(json_idea)})
    # print('invoked chain')
    return response.content

def update_google_sheet(date_str, play_text):
    return

def process_ideas(df):
    # ai_output = ""  # Initialize ai_output
    # date_str = datetime.datetime.now().strftime("%Y-%m-%d")
    # failure_step = 'last 60'

    # try:
    #     df_last60 = extract_last_60_days(file_path)
        
    #     failure_step = 'csv conversion'
    #     base64_data = csv_to_base64(df_last60)
        
    #     failure_step = 'call agent'
    #     ai_output = call_agent(base64_data)
        
    #     failure_step = 'append to sheet'
    #     append_to_google_sheet(date_str, ai_output)
    #     print(f"Successfully processed {os.path.basename(file_path)}")
    #     return ai_output  # No error
    # except Exception as e:
    #     error_message = f"Error processing {os.path.basename(file_path)} on {date_str}: {e}\nAI Output: {ai_output}\n\n"
    #     print(error_message)
    #     print(f'step failed: {failure_step}')
    #     print('=====')
    #     return error_message
    return

In [4]:
ideas = get_trade_ideas()
ideas

2025-07-25 18:02:48.828721


,date,play,date of outcome,outcome
0,2025-06-27,Ticker: $IONQ\nTrade Type: Swing\nBias: Bullis...,NaN,NaN
1,2025-07-11,Ticker: $PSIX\nTrade Type: Swing\nBias: Bullis...,NaN,NaN
2,2025-07-11,\nTicker: $PLAY\nTrade Type: Swing\nBias: Bull...,NaN,NaN
3,2025-07-11,Ticker: $NDAQ\nTrade Type: Swing\nBias: Bullis...,NaN,NaN
4,2025-07-11,Ticker: $GVA\nTrade Type: Swing\nBias: Bullish...,NaN,NaN
5,2025-07-11,Ticker: $FORM\nTrade Type: Swing\nBias: Bullis...,NaN,NaN
6,2025-07-11,Ticker: $FLS\nTrade Type: Swing\nBias: Bullish...,NaN,NaN
7,2025-07-11,Ticker: $FLR\nTrade Type: Swing\nBias: Bullish...,NaN,NaN
8,2025-07-11,Ticker: $DOOO\nTrade Type: Swing\nBias: Bullis...,NaN,NaN
9,2025-07-11,Ticker: $C\nTrade Type: Swing\nBias: Bullish\n...,NaN,NaN


In [15]:
trades = []

for (i, row) in ideas[:2].iterrows():
    # print(i)
    # print(row['play'])
    # idea_obj_raw = call_idea_parse_agent(row['play'])
    # idea_obj = idea_obj_raw.replace('```json','').replace('```','')
    # json_idea = json.loads(idea_obj)
    # # print(json_idea, json_idea['ticker'])
    # price_history = extract_last_60_days(json_idea['ticker'])
    # price_history_str = csv_to_base64(price_history)

    # trades.append([row['date'],json_idea,price_history_str])

    analysis_raw = call_analysis_agent(row['play'], row['date'])
    # analysis_gb = analysis_raw.groupby(['trade_entered', 'sl_hit', 'tp_hit']).agg({'Date':min})
    
    print(analysis_raw)
    # analysis_obj = analysis_raw.replace('```json','').replace('```','')
    # json_analysis = json.loads(analysis_obj)
    # print(json_analysis)
    # trades.append([row['date'],row['play'],json_analysis['date of outcome'],json_analysis['outcome']])

trades

[*********************100%***********************]  1 of 1 completed
C:\Users\Max\AppData\Local\Temp\ipykernel_12120\3364946765.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recent_prices['entry'] = json_idea['entry']
C:\Users\Max\AppData\Local\Temp\ipykernel_12120\3364946765.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recent_prices['sl'] = json_idea['stop loss']
C:\Users\Max\AppData\Local\Temp\ipykernel_12120\3364946765.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy o

```json
{
  "outcome": "SL was hit",
  "date of outcome": "2023-10-26"
}
```


[*********************100%***********************]  1 of 1 completed
C:\Users\Max\AppData\Local\Temp\ipykernel_12120\3364946765.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recent_prices['entry'] = json_idea['entry']
C:\Users\Max\AppData\Local\Temp\ipykernel_12120\3364946765.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recent_prices['sl'] = json_idea['stop loss']
C:\Users\Max\AppData\Local\Temp\ipykernel_12120\3364946765.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy o

```json
{
  "outcome": "SL was hit",
  "date of outcome": "2023-10-26"
}
```


[]